In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras import backend as K
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

Using TensorFlow backend.


In [2]:
test_df = pd.read_csv('./data/test.csv')
train_df = pd.read_csv('./data/train.csv')

In [3]:
Y_train_df = train_df['label']
#Y_train_df = pd.get_dummies(Y_train_df).values
#Y_train_df = Y_train_df.values.reshape(-1, 1)
#Y_train_df = standard_scaler.fit_transform(Y_train_df)
#Y_train_df = keras.utils.np_utils.to_categorical(Y_train_df)
#Y_train_df = Y_train_df.reshape(-1, 10, 1)
print(Y_train_df.shape)
X_train_df = train_df.drop(columns="label")
#X_train_df = standard_scaler.fit_transform(X_train_df)
#X_train_df = keras.utils.np_utils.to_categorical(X_train_df)
print(X_train_df.shape)

(60000,)
(60000, 784)


In [4]:
X_test_df = test_df
#X_train_df = standard_scaler.fit_transform(X_train_df)
#X_train_df = keras.utils.np_utils.to_categorical(X_train_df)
print(X_test_df.shape)

(5000, 785)


In [5]:
np.random.seed(7)
# split into 67% for train and 33% for test
X_train, X_valid, y_train, y_valid = train_test_split(X_train_df, Y_train_df, test_size=0.15, random_state=7)

In [6]:
X_test = test_df.drop('id', axis = 1)
# X_test = standard_scaler.fit_transform(X_test)
y_test = test_df['id']
# y_test = keras.utils.np_utils.to_categorical(y_test)
y_test

0          0
1          1
2          2
3          3
4          4
        ... 
4995    4995
4996    4996
4997    4997
4998    4998
4999    4999
Name: id, Length: 5000, dtype: int64

In [7]:
x_train = X_train.values
x_valid = X_valid.values
y_train = y_train.values
y_valid = y_valid.values
x_test = X_test.values

In [8]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_valid = x_valid.reshape(x_valid.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_valid = x_valid.reshape(x_valid.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [9]:
x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_valid /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_valid.shape[0], 'test samples')

x_train shape: (51000, 28, 28, 1)
51000 train samples
9000 test samples


In [10]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

In [13]:
def get_model(num_classes, input_shape):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.25))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    return model

In [14]:
model = get_model(num_classes, input_shape)

W1028 19:15:27.803161 10536 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1028 19:15:27.832146 10536 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1028 19:15:27.837140 10536 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1028 19:15:27.903099 10536 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1028 19:15:27.908097 10536 deprecation_wrapper.py:119] From C:\Users\user\Anaconda

In [ ]:
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_valid, y_valid))

W1028 19:15:30.976417 10536 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 51000 samples, validate on 9000 samples
Epoch 1/12
51000/51000 [==============================] - 378s 7ms/step - loss: 0.2066 - acc: 0.9365 - val_loss: 0.0533 - val_acc: 0.9837
Epoch 2/12
51000/51000 [==============================] - 320s 6ms/step - loss: 0.0606 - acc: 0.9818 - val_loss: 0.0351 - val_acc: 0.9897
Epoch 3/12
36992/51000 [====================>.........] - ETA: 1:45 - loss: 0.0429 - acc: 0.9875

In [ ]:
answer = model.predict(x_test)

In [ ]:
answer = answer.astype(int)

In [ ]:
y = np.argmax(answer, axis=-1)
y

In [ ]:
frames_aswer = pd.DataFrame({"ImageId": pd.DataFrame(y).index.values+1,
                             "Label": y
                            })
frames_aswer

In [ ]:
frames_aswer.to_csv('./data/answer_cnn.csv', index=False)